# MGTA 463 Fraud Analytics
# Homework 8

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import datetime
from datetime import datetime, timedelta
import scipy.stats as sps
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings("ignore")

In [2]:
ct = pd.read_csv("df2.csv")
ct['Date'] = pd.to_datetime(ct['Date'])

In [3]:
ct

,Unnamed: 0,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,...,card_desc_count_0_by_30,card_desc_count_1_by_7,card_desc_count_1_by_14,card_desc_count_1_by_30,merch_desc_count_0_by_7,merch_desc_count_0_by_14,merch_desc_count_0_by_30,merch_desc_count_1_by_7,merch_desc_count_1_by_14,merch_desc_count_1_by_30
0,0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
1,1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
2,2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
3,3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
4,4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96748,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333
96393,96749,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,...,0.016667,0.142857,0.035714,0.016667,0.142857,0.014286,0.004762,0.142857,0.014286,0.004762
96394,96750,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,...,0.033333,0.142857,0.071429,0.033333,0.095238,0.040816,0.006667,0.095238,0.040816,0.006667
96395,96751,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,...,0.011111,0.142857,0.071429,0.011111,0.023810,0.005495,0.000725,0.095238,0.021978,0.002899


### Target Encoded Variable

In [4]:
nonoot = ct.loc[(ct.Date <= "2010-08-31") & (ct.Date > "2010-01-10"),:]

In [5]:
# Proba DOW
ct["DOW"] = ct.Date.dt.dayofweek
nonoot["DOW"] = nonoot.Date.dt.dayofweek
proba_DOW = nonoot.groupby("DOW")["Fraud"].value_counts(normalize=True).unstack()[1]
ct["Proba DOW"] = [proba_DOW[DOW] for DOW in ct["DOW"]]

In [6]:
# Proba state
proba_state = 1 - nonoot.groupby("Merch state")["Fraud"].value_counts(normalize=True).unstack()[0]
ct["Proba state"] = [proba_state[state] for state in ct["Merch state"]]

In [7]:
ct = ct.drop(["Amount", "Recnum", "Merch description",
              "Merch state", "Merch zip", "Transtype", "DOW"], axis=1)

In [8]:
# add in random number
ct["RND"] = np.random.choice(2, ct.shape[0])

In [9]:
# remove oot
ct = ct.loc[(ct.Date <= "2010-08-31") & (ct.Date > "2010-01-10"),:]
ct = ct.drop(["Date"], axis=1)

### Filtering

In [10]:
KSFDR = pd.DataFrame(columns=["ks","FDR"], index=ct.columns)
goods = ct.loc[ct["Fraud"] == 0]
bads = ct.loc[ct["Fraud"] == 1]
numgoods = goods.shape[0]
numbads = bads.shape[0]

In [11]:
i = 0
topRows = int(round(len(ct) * .03))
for col in ct:
    # calculate univariate KS
    KSFDR["ks"][i] = sps.ks_2samp(goods[col], bads[col])[0]
    
    # calculate univariate FDR@3%
    temp = ct.sort_values(col, ascending=False)
    temp1 = temp.head(topRows)
    temp2 = temp.tail(topRows)
    needed1 = temp1.loc[:,"Fraud"]
    needed2 = temp2.loc[:,"Fraud"]
    FDR1 = sum(needed1) / numbads
    FDR2 = sum(needed2) / numbads
    FDRate = np.maximum(FDR1,FDR2)
    KSFDR["FDR"][i] = FDRate
    
    i += 1

In [14]:
# calculate rank
KSFDR["rank_ks"] = KSFDR["ks"].rank(ascending=True)
KSFDR["rank_FDR"] = KSFDR["FDR"].rank(ascending=True)
KSFDR["average_rank"] = (KSFDR["rank_ks"] + KSFDR["rank_FDR"]) / 2
KSFDR.sort_values(by=["average_rank"], ascending=False, inplace=True)
KSFDR = KSFDR.loc[KSFDR.index != "Fraud"]
KSFDR = KSFDR.iloc[:80]

In [16]:
KSFDR = KSFDR.rename(columns={"ks":"KS","FDR":"FDR @3%","rank_ks":"KS Rank","rank_FDR":"FDR Rank","average_rank":"Average Rank"})

In [18]:
KSFDR.to_csv("vars_80.csv")

In [18]:
KSFDR.to_csv("Filtered Variables.csv")

In [19]:
# top 80 variables

vars_80 = KSFDR.index
ct_80 = ct[vars_80]

### Wrapping

In [20]:
# function for calculating FDR for forward selection
def fdr(classifier, x, y, cutoff=.03):
    # calculate FDR score for the given classifier on dataset x and y with cutoff value
    # get the probability list from the given classifier
    return fdr_prob(y, classifier.predict_proba(x), cutoff)

In [21]:
def fdr_prob(y, y_prob, cutoff=.03):
    if len(y_prob.shape) != 1: # sometimes the proba list can contain many columns, one for each category
        y_prob = y_prob[:,-1:] # only the last one (fraud_label==1) is used here.
    num_fraud = len(y[y == 1]) # count the total number of frauds
    
    # sort the proba list from high to low while retain the true (not predicted) fraud label
    sorted_prob = np.asarray(sorted(zip(y_prob, y), key=lambda x: x[0], reverse=True))
    cutoff_bin = sorted_prob[0:int(len(y) * cutoff), 1:] # 3% cutoff
    
    # return the FDR score (#fraud_in_cutoff / #total_fraud)
    return len(cutoff_bin[cutoff_bin == 1]) / num_fraud

In [22]:
# forward selection
nfeatures = ct_80.shape[1]
clf = RandomForestClassifier(
    n_estimators=5,
    n_jobs=-1
)
sfs = SFS(
    clf,
    k_features=nfeatures,
    forward=True,
    verbose=2,
    scoring=fdr,
    cv=2,
    n_jobs=-1,
)
sfs.fit(ct_80,ct.Fraud)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   19.0s finished

[2021-06-08 17:22:43] Features: 1/80 -- score: 0.3841962304150518[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  79 out of  79 | elapsed:   17.3s finished

[2021-06-08 17:23:01] Features: 2/80 -- score: 0.528089701546921[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  78 out of  78 | elapsed:   15.3s finished

[2021-06-08 17:23:16] Features: 3/80 -- score: 0.5755656033654643[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  77 out of

SequentialFeatureSelector(cv=2,
                          estimator=RandomForestClassifier(n_estimators=5,
                                                           n_jobs=-1),
                          k_features=80, n_jobs=-1,
                          scoring=<function fdr at 0x7f8acf42a700>, verbose=2)

In [23]:
vars_FS = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
feature_idx = vars_FS.feature_idx 

In [24]:
idx_30 = [feature_idx[1][0]]
for step in range(2,31):
    for idx in feature_idx[step]:
        if idx not in feature_idx[step-1]:
            idx_30.append(idx)

In [25]:
vars_30 = [vars_80[idx] for idx in idx_30]
vars_30 = pd.DataFrame({"Variable" : vars_30}, index=range(1,31))
vars_30.to_csv("vars_30.csv")

In [26]:
X = ct[vars_30["Variable"]]
y = ct["Fraud"]

In [27]:
ct_orig = pd.read_csv("card transactions.csv")

In [ ]:
ct_orig